## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import os
import sys
sys.path.append(os.path.join(os.path.dirname(sys.path[0])))
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
#file_to_load=os.path.join(os.getcwd(), "WeatherPy_Database.csv")
file_to_load=file_to_load="/Users/Mark/Desktop/Analysis Projects/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
city_data_df = pd.read_csv(file_to_load)


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your destination?"))
max_temp=float(input("What is the maximum temperature you would like for your destination?"))

What is the minimum temperature you would like for your destination?65
What is the maximum temperature you would like for your destination?77


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                     (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,4,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
1,6,Rikitea,PF,76.73,clear sky,-23.12,-134.97,Pension Maro'i
2,9,Los Llanos De Aridane,ES,68.00,broken clouds,28.66,-17.92,Valle Aridane
3,11,Port Alfred,ZA,71.44,few clouds,-33.59,26.89,The Halyards Hotel
4,13,Cape Town,ZA,73.00,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     3
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferable_cities=preferred_cities_df.dropna()

In [10]:
preferable_cities.isnull().sum()

City_ID        0
City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df =preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
1,Rikitea,PF,76.73,clear sky,-23.12,-134.97,
2,Los Llanos De Aridane,ES,68.00,broken clouds,28.66,-17.92,
3,Port Alfred,ZA,71.44,few clouds,-33.59,26.89,
4,Cape Town,ZA,73.00,few clouds,-33.93,18.42,
5,Karachi,PK,73.40,haze,24.91,67.08,
6,Bambous Virieux,MU,75.99,shower rain,-20.34,57.76,
7,Bobon,PH,75.88,broken clouds,12.52,124.56,
8,Saldanha,ZA,71.60,overcast clouds,-33.01,17.94,
10,Grand Gaube,MU,75.99,light rain,-20.01,57.66,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
counters=1
record_count=1
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels=requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [13]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
1,Rikitea,PF,76.73,clear sky,-23.12,-134.97,Pension Maro'i
2,Los Llanos De Aridane,ES,68.00,broken clouds,28.66,-17.92,Valle Aridane
3,Port Alfred,ZA,71.44,few clouds,-33.59,26.89,The Halyards Hotel
4,Cape Town,ZA,73.00,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
...,...,...,...,...,...,...,...
103,Guaimaro,CU,69.80,broken clouds,21.05,-77.35,La Constitución de Guáimaro
105,Puerto Madryn,AR,74.52,clear sky,-42.77,-65.04,Hotel Tolosa
106,Buraydah,SA,71.60,clear sky,26.33,43.98,Mövenpick Hotel Qassim
108,Andros Town,BS,68.00,broken clouds,24.70,-77.77,Hanks place


In [14]:
hotel_df.dropna()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
1,Rikitea,PF,76.73,clear sky,-23.12,-134.97,Pension Maro'i
2,Los Llanos De Aridane,ES,68.00,broken clouds,28.66,-17.92,Valle Aridane
3,Port Alfred,ZA,71.44,few clouds,-33.59,26.89,The Halyards Hotel
4,Cape Town,ZA,73.00,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
...,...,...,...,...,...,...,...
103,Guaimaro,CU,69.80,broken clouds,21.05,-77.35,La Constitución de Guáimaro
105,Puerto Madryn,AR,74.52,clear sky,-42.77,-65.04,Hotel Tolosa
106,Buraydah,SA,71.60,clear sky,26.33,43.98,Mövenpick Hotel Qassim
108,Andros Town,BS,68.00,broken clouds,24.70,-77.77,Hanks place


In [21]:
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [22]:
#7. Drop all of the empty rows in hotel_df
clean_hotel_df=hotel_df.dropna()

In [23]:
clean_hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [26]:
# 8a. Create the output File (CSV)
#output_data_file= "Vacation_Search/WeatherPy_vacation"
output_data_file=os.path.join(os.getcwd(),"WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [5]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


NameError: name 'locations' is not defined